# Möbius Grid – v12 **real‑safe damping**
Integrates `damped_breath_real` so fractional powers never create complex numbers.
Searches gap_scale 20 / spacing 3 with φ‑micro grid, k‑sweep, λ‑sweep.

In [1]:

import mpmath as mp, pandas as pd, itertools, math
mp.mp.dps=260
ALPHA_INV=mp.mpf('137.035999084'); ALPHA=1/ALPHA_INV
PHI=(1+mp.sqrt(5))/2


In [2]:

def primes_upto(n=30000):
    sieve=[True]*(n+1); sieve[0:2]=[False,False]
    for i in range(2,int(n**0.5)+1):
        if sieve[i]:
            sieve[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [i for i,v in enumerate(sieve) if v]

PRIMES=primes_upto()
def prime_gaps(cnt): return [PRIMES[i+1]-PRIMES[i] for i in range(cnt)]


In [3]:

def build_tail(bright_len=400, spacing=3, gap_scale=20):
    gaps=[round(g/gap_scale,4) for g in prime_gaps(bright_len//spacing)]
    tail=[]; gi=iter(gaps)
    for i in range(bright_len):
        tail.append(1)
        if (i+1)%spacing==0: tail.append(next(gi,1))
    return tail

def cf_to_real(coefs):
    v=mp.mpf(0)
    for a in reversed(coefs): v=1/(a+v)
    return v


In [4]:

def breath_once(x,tiny,phi):
    x = phi*x
    x -= tiny
    x = 1/x/9
    x = 1/x
    return x/9

def damped_breath_real(x,tiny,phi,lam=None):
    x = breath_once(x,tiny,phi)
    if lam is None: return x
    sign = mp.sign(x)
    return sign * mp.power(abs(x), lam)


In [5]:

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n: a,b=b,b+a
    return b[:n]
def fib_fraction(bits):
    s=mp.mpf(0)
    for i,b in enumerate(bits):
        if b=='1':
            s += mp.power(2,-(i+1))
    return s

F64=fib_fraction(fib_word_bits(64))
F128=fib_fraction(fib_word_bits(128))


In [6]:

phi_vals=[mp.mpf('0.003944')+i*1e-7 for i in range(-4,5)]
k_vals=[3.8,4.0,4.2,4.3,4.4,4.5]
lam_vals=[None,0.2,0.3,0.4,0.5]
tail=build_tail()
seed=cf_to_real(tail)
rows=[]
for bits in (64,128,256):
    for k in k_vals:
        tiny=ALPHA_INV/mp.power(10,k)
        for phi in phi_vals:
            for lam in lam_vals:
                x=damped_breath_real(seed+tiny,tiny,phi,lam)
                rel=float(abs((x-ALPHA)/ALPHA))
                rows.append(dict(bits=bits,k=k,phi=float(phi),lam=lam or 0,rel_err=rel,
                                 fib64=abs(x-F64)<1e-5,fib128=abs(x-F128)<1e-5))
df=pd.DataFrame(rows).sort_values('rel_err')
print('Best 20 rows:')
display(df.head(20))


Best 20 rows:


,bits,k,phi,lam,rel_err,fib64,fib128
535,128,4.5,0.003944,0.0,1.277307,False,False
265,64,4.5,0.003944,0.0,1.277307,False,False
805,256,4.5,0.003944,0.0,1.277307,False,False
530,128,4.5,0.003944,0.0,1.277315,False,False
260,64,4.5,0.003944,0.0,1.277315,False,False
800,256,4.5,0.003944,0.0,1.277315,False,False
255,64,4.5,0.003944,0.0,1.277323,False,False
525,128,4.5,0.003944,0.0,1.277323,False,False
795,256,4.5,0.003944,0.0,1.277323,False,False
250,64,4.5,0.003944,0.0,1.277331,False,False
